# Lakeflow Pipelines: Bronze → Silver → Gold 🥉🥈🥇

Welcome! This demo will teach you how to build declarative data pipelines using **Lakeflow** (Delta Live Tables).

---

## 🎯 What You'll Learn

By the end of this demo, you will be able to:

1. ✅ Understand Lakeflow (Delta Live Tables) concepts
2. ✅ Differentiate between streaming tables and materialized views
3. ✅ Create bronze layer with streaming tables
4. ✅ Build silver layer with data quality transformations
5. ✅ Create gold layer with materialized views
6. ✅ Configure and run a Lakeflow pipeline
7. ✅ View pipeline results and lineage
8. ✅ Understand declarative vs imperative pipelines

---

## 🌊 What is Lakeflow?

**Lakeflow** (formerly Delta Live Tables / DLT) is a declarative framework for building reliable data pipelines.

**Key concepts:**
* **Declarative** - Define WHAT you want, not HOW to do it
* **Automatic** - Handles dependencies, retries, scaling
* **Reliable** - Built-in data quality and monitoring
* **Incremental** - Processes only new data
* **Managed** - No infrastructure management

**Traditional pipeline vs Lakeflow:**

| Traditional | Lakeflow |
|-------------|----------|
| Write orchestration code | Declare tables |
| Handle dependencies manually | Automatic dependency resolution |
| Manage incremental logic | Built-in incremental processing |
| Write error handling | Automatic retries |
| Monitor manually | Built-in monitoring |

---

## 📊 Demo Scenario: Wanderbricks Analytics

You'll build a pipeline for **Wanderbricks** booking data:

**Pipeline flow:**
```
📁 Raw Data (samples.wanderbricks)
    ↓
🥉 Bronze: Streaming tables (raw ingestion)
    ↓
🥈 Silver: Streaming tables (cleaned, validated)
    ↓
🥇 Gold: Materialized views (aggregated metrics)
```

**Data sources:**
* Bookings data
* Properties data
* Reviews data

---

**Let's get started!** 🚀

## Part 1: Lakeflow Core Concepts 📚

Before building, let's understand the key concepts.

---

### 🌊 Streaming Tables vs Materialized Views

**Lakeflow has two table types:**

---

### **1. Streaming Tables**

**What they are:**
* Process data incrementally as it arrives
* Append-only by default
* Maintain state for incremental processing
* Always up-to-date

**When to use:**
* ✅ Bronze layer (raw data ingestion)
* ✅ Silver layer (incremental transformations)
* ✅ Real-time or near-real-time data
* ✅ Append-only data (logs, events, transactions)
* ✅ When you need low latency

**Syntax:**
```python
@dlt.table
def bronze_bookings():
    return spark.readStream.table("samples.wanderbricks.bookings")
```

---

### **2. Materialized Views**

**What they are:**
* Recompute entire result on each update
* Can handle updates and deletes
* Batch processing
* Optimized for aggregations

**When to use:**
* ✅ Gold layer (aggregations)
* ✅ Complex joins
* ✅ Aggregations (GROUP BY)
* ✅ When you need complete refresh
* ✅ Smaller result sets

**Syntax:**
```python
@dlt.view
def gold_daily_revenue():
    return dlt.read("silver_bookings").groupBy("date").agg(...)
```

---

### **Key Differences:**

| Feature | Streaming Table | Materialized View |
|---------|----------------|-------------------|
| **Processing** | Incremental | Full refresh |
| **Updates** | Append-only | Supports updates/deletes |
| **Latency** | Low (real-time) | Higher (batch) |
| **Use case** | Bronze, Silver | Gold (aggregations) |
| **State** | Maintains checkpoints | Stateless |
| **Best for** | Large data volumes | Aggregated results |

---

**Rule of thumb:**
* 🥉 **Bronze:** Streaming tables (ingest raw data)
* 🥈 **Silver:** Streaming tables (incremental cleaning)
* 🥇 **Gold:** Materialized views (aggregations)

---

### 📝 Declarative Pipeline Syntax

**Lakeflow uses Python decorators to declare tables:**

---

### **Basic syntax:**

```python
import dlt
from pyspark.sql.functions import *

# Streaming table
@dlt.table(
    name="table_name",
    comment="Description of the table"
)
def table_name():
    return spark.readStream.table("source_table")

# Materialized view
@dlt.view(
    name="view_name",
    comment="Description of the view"
)
def view_name():
    return dlt.read("upstream_table").groupBy(...).agg(...)
```

---

### **Key components:**

**1. Import dlt module:**
```python
import dlt
```

**2. Decorator:**
* `@dlt.table` - Creates a streaming table
* `@dlt.view` - Creates a materialized view

**3. Function:**
* Function name becomes table name (if not specified)
* Returns a DataFrame
* Can use PySpark transformations

**4. Reading data:**
* `spark.readStream.table()` - Read from source (streaming)
* `dlt.read()` - Read from another DLT table
* `spark.read.table()` - Read batch data

---

### **Dependencies are automatic:**

```python
# Bronze
@dlt.table
def bronze_bookings():
    return spark.readStream.table("samples.wanderbricks.bookings")

# Silver (depends on bronze)
@dlt.table
def silver_bookings():
    return dlt.read_stream("bronze_bookings").filter(...)  # Automatic dependency!

# Gold (depends on silver)
@dlt.view
def gold_revenue():
    return dlt.read("silver_bookings").groupBy(...).agg(...)  # Automatic dependency!
```

**Lakeflow automatically:**
* ✅ Detects dependencies
* ✅ Runs tables in correct order
* ✅ Handles failures and retries
* ✅ Manages incremental processing

---

## Part 2: Bronze Layer - Raw Data Ingestion 🥉

The bronze layer ingests raw data using **streaming tables**.

**Bronze characteristics:**
* Ingest data as-is (no transformations)
* Streaming tables for incremental processing
* Append-only
* Full audit trail

---

In [0]:
# Import required modules
import dlt
from pyspark.sql.functions import *

# ============================================
# BRONZE LAYER: Raw Data Ingestion
# ============================================

@dlt.table(
    name="bronze_bookings",
    comment="Raw booking data from Wanderbricks - Bronze layer"
)
def bronze_bookings():
    """
    Ingest raw booking data as streaming table.
    No transformations - data as-is from source.
    """
    return (
        spark.readStream
            .table("samples.wanderbricks.bookings")
    )

@dlt.table(
    name="bronze_properties",
    comment="Raw property data from Wanderbricks - Bronze layer"
)
def bronze_properties():
    """
    Ingest raw property data as streaming table.
    """
    return (
        spark.readStream
            .table("samples.wanderbricks.properties")
    )

@dlt.table(
    name="bronze_reviews",
    comment="Raw review data from Wanderbricks - Bronze layer"
)
def bronze_reviews():
    """
    Ingest raw review data as streaming table.
    """
    return (
        spark.readStream
            .table("samples.wanderbricks.reviews")
    )

### 💡 Bronze Layer - Key Points

**What we did:**

1. **Created 3 streaming tables:**
   * `bronze_bookings` - Booking transactions
   * `bronze_properties` - Property listings
   * `bronze_reviews` - Customer reviews

2. **Used `@dlt.table` decorator:**
   * Declares a streaming table
   * Automatically managed by Lakeflow
   * Incremental processing

3. **Used `spark.readStream.table()`:**
   * Reads from source tables as streams
   * Processes incrementally
   * Tracks progress automatically

**Why streaming tables for bronze?**
* ✅ Incremental ingestion (only new data)
* ✅ Low latency
* ✅ Efficient for large data volumes
* ✅ Automatic checkpoint management
* ✅ Append-only semantics

**What Lakeflow handles automatically:**
* ✅ Checkpoint management
* ✅ Incremental processing
* ✅ Schema evolution
* ✅ Failure recovery
* ✅ Backpressure

---

**💡 Tip:** Bronze layer is always streaming tables - ingest raw data as-is

## Part 3: Silver Layer - Cleaned & Validated Data 🥈

The silver layer cleans and validates data using **streaming tables**.

**Silver characteristics:**
* Clean and validate data
* Remove duplicates
* Filter invalid records
* Join related tables
* Still streaming (incremental)

---

In [0]:
# ============================================
# SILVER LAYER: Cleaned and Validated Data
# ============================================

@dlt.table(
    name="silver_bookings",
    comment="Cleaned and validated booking data - Silver layer"
)
def silver_bookings():
    """
    Clean bronze bookings:
    - Filter confirmed bookings only
    - Remove invalid data (null amounts, invalid dates)
    - Add calculated columns
    - Deduplicate
    """
    return (
        dlt.read_stream("bronze_bookings")
        # Data quality filters
        .filter(col("status") == "confirmed")
        .filter(col("total_amount").isNotNull())
        .filter(col("total_amount") > 0)
        .filter(col("check_in") < col("check_out"))  # Valid date range
        # Add calculated columns
        .withColumn("nights", datediff(col("check_out"), col("check_in")))
        .withColumn("booking_date", to_date(col("created_at")))
        .withColumn("booking_year", year(col("created_at")))
        .withColumn("booking_month", month(col("created_at")))
        # Select relevant columns
        .select(
            "booking_id",
            "user_id",
            "property_id",
            "check_in",
            "check_out",
            "nights",
            "guests_count",
            "total_amount",
            "status",
            "booking_date",
            "booking_year",
            "booking_month",
            "created_at"
        )
    )

@dlt.table(
    name="silver_properties",
    comment="Cleaned property data - Silver layer"
)
def silver_properties():
    """
    Clean bronze properties:
    - Filter valid properties
    - Standardize property types
    """
    return (
        dlt.read_stream("bronze_properties")
        # Data quality filters
        .filter(col("property_type").isNotNull())
        .filter(col("base_price").isNotNull())
        .filter(col("base_price") > 0)
        # Select relevant columns
        .select(
            "property_id",
            "host_id",
            "destination_id",
            "title",
            "property_type",
            "base_price",
            "max_guests",
            "bedrooms",
            "bathrooms",
            "created_at"
        )
    )

@dlt.table(
    name="silver_reviews",
    comment="Cleaned review data - Silver layer"
)
def silver_reviews():
    """
    Clean bronze reviews:
    - Filter non-deleted reviews
    - Filter valid ratings
    """
    return (
        dlt.read_stream("bronze_reviews")
        # Data quality filters
        .filter(col("is_deleted") == False)
        .filter(col("rating").isNotNull())
        .filter((col("rating") >= 1.0) & (col("rating") <= 5.0))
        # Add calculated columns
        .withColumn("review_date", to_date(col("created_at")))
        .withColumn("review_year", year(col("created_at")))
        .withColumn("review_month", month(col("created_at")))
        # Select relevant columns
        .select(
            "review_id",
            "booking_id",
            "property_id",
            "user_id",
            "rating",
            "comment",
            "review_date",
            "review_year",
            "review_month",
            "created_at"
        )
    )

### 💡 Silver Layer - Key Points

**What we did:**

1. **Created 3 streaming tables:**
   * `silver_bookings` - Cleaned bookings
   * `silver_properties` - Cleaned properties
   * `silver_reviews` - Cleaned reviews

2. **Used `dlt.read_stream()`:**
   * Reads from bronze streaming tables
   * Maintains streaming semantics
   * Incremental processing continues

3. **Applied data quality rules:**
   * Filter invalid data (nulls, negatives, invalid ranges)
   * Remove soft-deleted records
   * Validate business rules

4. **Added calculated columns:**
   * `nights` - Duration of stay
   * `booking_date` - Date portion of timestamp
   * `booking_year`, `booking_month` - For aggregations

**Why streaming tables for silver?**
* ✅ Incremental processing (efficient)
* ✅ Low latency
* ✅ Handles large data volumes
* ✅ Maintains data lineage

**Data quality patterns:**
```python
# Filter nulls
.filter(col("column").isNotNull())

# Filter invalid values
.filter(col("amount") > 0)

# Filter by status
.filter(col("status") == "confirmed")

# Validate ranges
.filter((col("rating") >= 1.0) & (col("rating") <= 5.0))
```

---

**💡 Tip:** Silver layer filters bad data and adds business logic

In [0]:
# ============================================
# SILVER LAYER: Enriched Data with JOINs
# ============================================

@dlt.table(
    name="silver_bookings_enriched",
    comment="Bookings enriched with property details - Silver layer"
)
def silver_bookings_enriched():
    """
    Join bookings with properties to create enriched dataset.
    This is still a streaming table for incremental processing.
    """
    bookings = dlt.read_stream("silver_bookings")
    properties = dlt.read_stream("silver_properties")
    
    return (
        bookings
        .join(
            properties,
            bookings.property_id == properties.property_id,
            "left"
        )
        .select(
            bookings["*"],
            properties["property_type"],
            properties["title"].alias("property_name"),
            properties["base_price"],
            properties["bedrooms"],
            properties["bathrooms")
        )
    )

### 🔗 Silver Layer - JOINs in Streaming

**What we did:**

1. **Created enriched streaming table:**
   * `silver_bookings_enriched` - Bookings + property details

2. **Joined two streaming tables:**
   * Read both as streams with `dlt.read_stream()`
   * Perform stream-stream JOIN
   * Result is still a streaming table

3. **Selected columns from both tables:**
   * All booking columns
   * Property type, name, price, bedrooms, bathrooms

**Stream-stream JOINs:**
* ✅ Both sides are streaming
* ✅ Incremental processing
* ✅ Lakeflow manages state
* ✅ Automatic watermarking

**JOIN patterns in DLT:**
```python
# Stream-stream JOIN
left_stream = dlt.read_stream("table1")
right_stream = dlt.read_stream("table2")
result = left_stream.join(right_stream, "key")

# Stream-batch JOIN (dimension lookup)
stream = dlt.read_stream("fact_table")
dimension = dlt.read("dimension_table")  # Batch read
result = stream.join(dimension, "key")
```

---

**💡 Tip:** Use stream-stream JOINs for fact tables, stream-batch for dimension lookups

## Part 4: Gold Layer - Aggregated Metrics 🥇

The gold layer creates business metrics using **materialized views**.

**Gold characteristics:**
* Aggregated data (GROUP BY)
* Business-ready metrics
* Materialized views (full refresh)
* Optimized for analytics
* Smaller result sets

---

In [0]:
# ============================================
# GOLD LAYER: Aggregated Business Metrics
# ============================================

@dlt.view(
    name="gold_daily_bookings",
    comment="Daily booking metrics - Gold layer"
)
def gold_daily_bookings():
    """
    Daily aggregated booking metrics.
    Uses materialized view for full refresh aggregations.
    """
    return (
        dlt.read("silver_bookings_enriched")
        .groupBy("booking_date", "property_type")
        .agg(
            count("booking_id").alias("booking_count"),
            sum("total_amount").alias("total_revenue"),
            avg("total_amount").alias("avg_booking_value"),
            sum("nights").alias("total_nights"),
            avg("nights").alias("avg_nights"),
            countDistinct("user_id").alias("unique_customers"),
            countDistinct("property_id").alias("unique_properties")
        )
        .orderBy("booking_date", "property_type")
    )

@dlt.view(
    name="gold_property_performance",
    comment="Property performance metrics - Gold layer"
)
def gold_property_performance():
    """
    Aggregated metrics by property type.
    """
    return (
        dlt.read("silver_bookings_enriched")
        .groupBy("property_type")
        .agg(
            count("booking_id").alias("total_bookings"),
            sum("total_amount").alias("total_revenue"),
            avg("total_amount").alias("avg_revenue_per_booking"),
            avg("nights").alias("avg_stay_duration"),
            avg("guests_count").alias("avg_guests"),
            countDistinct("property_id").alias("property_count")
        )
        .orderBy(desc("total_revenue"))
    )

@dlt.view(
    name="gold_review_summary",
    comment="Review rating summary by property type - Gold layer"
)
def gold_review_summary():
    """
    Average ratings by property type and time period.
    Joins reviews with properties.
    """
    reviews = dlt.read("silver_reviews")
    properties = dlt.read("silver_properties")
    
    return (
        reviews
        .join(properties, "property_id", "left")
        .groupBy("property_type", "review_year", "review_month")
        .agg(
            count("review_id").alias("review_count"),
            avg("rating").alias("avg_rating"),
            min("rating").alias("min_rating"),
            max("rating").alias("max_rating")
        )
        .orderBy("review_year", "review_month", "property_type")
    )

### 💡 Gold Layer - Key Points

**What we did:**

1. **Created 3 materialized views:**
   * `gold_daily_bookings` - Daily metrics by property type
   * `gold_property_performance` - Overall property type performance
   * `gold_review_summary` - Review ratings over time

2. **Used `@dlt.view` decorator:**
   * Creates materialized views
   * Full refresh on each update
   * Optimized for aggregations

3. **Used `dlt.read()` (not read_stream):**
   * Batch read from silver tables
   * Appropriate for aggregations
   * Full table scan

4. **Applied aggregations:**
   * `count()`, `sum()`, `avg()`, `min()`, `max()`
   * `countDistinct()` for unique counts
   * `groupBy()` for dimensions

**Why materialized views for gold?**
* ✅ Aggregations require full data
* ✅ Smaller result sets (efficient full refresh)
* ✅ Optimized for analytics
* ✅ Simpler than streaming aggregations
* ✅ Perfect for dashboards

**Streaming table vs Materialized view for gold:**

| Use Case | Recommendation |
|----------|----------------|
| Simple aggregations | Materialized view |
| Large fact tables | Streaming table |
| Real-time metrics | Streaming table |
| Dashboard metrics | Materialized view |
| Complex joins | Materialized view |

---

**💡 Tip:** Gold layer typically uses materialized views for aggregations

## Part 5: Creating Your Lakeflow Pipeline 🛠️

Now that we've written the code, let's create the pipeline in the Databricks UI.

---

### 💾 Step 1: Save This Notebook

**Follow these steps:**

1. This notebook contains all your pipeline code
2. Click **"Save"** or press `Ctrl+S` / `Cmd+S`
3. Note the notebook path (you'll need it)
4. Example path: `/Users/your.email@company.com/Lakeflow Pipeline Fundamentals`

**What's in this notebook:**
* Bronze layer: 3 streaming tables
* Silver layer: 4 streaming tables (including enriched)
* Gold layer: 3 materialized views
* Total: 10 tables/views

✅ **Checkpoint:** Notebook is saved

---

**💡 Tip:** The notebook IS your pipeline definition

### 🏗️ Step 2: Create a New Pipeline

**Follow these steps:**

1. Click **"Workflows"** in the left sidebar
2. Click the **"Delta Live Tables"** tab
3. Click **"Create Pipeline"** button
4. A pipeline configuration form opens

**Pipeline configuration form:**

**General settings:**
* **Pipeline name:** "Wanderbricks Analytics Pipeline"
* **Product edition:** Advanced (or Core)
* **Pipeline mode:** Triggered (or Continuous)

**Source code:**
* **Notebook libraries:** Click "Add notebook"
* **Path:** Select this notebook (Lakeflow Pipeline Fundamentals)
* The notebook path should appear

**Destination:**
* **Target schema:** `main.default` (or create new schema)
* **Storage location:** Leave default (optional)

**Compute:**
* **Cluster mode:** Enhanced autoscaling (recommended)
* **Min workers:** 1
* **Max workers:** 5
* **Serverless:** Enable if available (recommended)

**Advanced settings (leave defaults for now):**
* Channel: Current
* Photon: Enabled
* Configuration: Empty

5. Click **"Create"** button

✅ **Checkpoint:** Pipeline created (not yet run)

---

**💡 Tip:** Serverless DLT is faster and easier - no cluster management

### ⚙️ Pipeline Configuration - Key Settings

**Pipeline mode:**

**Triggered mode:**
* Runs on-demand or on schedule
* Processes all available data
* Stops after completion
* Use for: Batch pipelines, scheduled jobs

**Continuous mode:**
* Runs continuously
* Processes data as it arrives
* Always running
* Use for: Real-time pipelines, streaming data

**For this demo:** Use **Triggered** mode

---

**Product edition:**

**Core:**
* Basic DLT features
* Streaming tables and views
* Good for development

**Pro:**
* Adds expectations (data quality)
* SLA monitoring
* Advanced features

**Advanced:**
* All Pro features
* Enhanced performance
* Production workloads

**For this demo:** **Advanced** (or Core if Advanced not available)

---

**Target schema:**
* Where tables will be created
* Example: `main.default`
* All DLT tables appear here
* Can query tables after pipeline runs

---

**Compute:**
* **Serverless** - Recommended, no cluster management
* **Enhanced autoscaling** - Automatically scales workers
* **Fixed size** - Specific number of workers

**For this demo:** **Serverless** (if available) or **Enhanced autoscaling**

---

### 🔍 Step 3: Review Pipeline Configuration

**Follow these steps:**

After creating the pipeline:

1. You're taken to the **pipeline details page**
2. Review the configuration:

**Pipeline details page shows:**
* 📝 **Pipeline name** - "Wanderbricks Analytics Pipeline"
* 💾 **Source notebook** - Path to this notebook
* 🎯 **Target schema** - Where tables are created
* 💻 **Compute** - Cluster configuration
* 📅 **Mode** - Triggered or Continuous

**Tabs available:**
* **Graph** - Visual pipeline lineage (we'll explore this)
* **Event log** - Execution history and logs
* **Settings** - Edit configuration

3. Click the **"Settings"** tab to review
4. You can edit any setting here
5. Click **"Save"** if you make changes

✅ **Checkpoint:** Pipeline configuration reviewed

---

**💡 Tip:** You can edit pipeline settings anytime from the Settings tab

## Part 6: Running Your Pipeline ▶️

Let's execute the pipeline and see it in action!

---

### ▶️ Step 4: Start the Pipeline

**Follow these steps:**

1. On the pipeline details page, click the **"Start"** button (top-right)
2. The pipeline begins execution
3. You'll see:
   * Status changes to "Starting"
   * Then "Running"
   * Progress indicators appear

**What happens when you start:**
* ✅ Lakeflow reads your notebook
* ✅ Parses all `@dlt.table` and `@dlt.view` declarations
* ✅ Builds dependency graph
* ✅ Provisions compute resources
* ✅ Executes tables in correct order
* ✅ Creates tables in target schema

**First run:**
* Takes longer (initial data load)
* Creates all tables from scratch
* Processes all historical data

**Subsequent runs:**
* Faster (incremental processing)
* Only processes new data
* Updates existing tables

✅ **Checkpoint:** Pipeline is running

---

**💡 Tip:** First run may take 5-10 minutes depending on data volume

### 📊 Step 5: Explore the Pipeline Graph

**Follow these steps:**

1. Click the **"Graph"** tab (should be default view)
2. You'll see a **visual representation** of your pipeline

**What the graph shows:**

**Nodes (tables/views):**
* 🥉 **Bronze tables** - Raw data sources
* 🥈 **Silver tables** - Cleaned data
* 🥇 **Gold views** - Aggregated metrics
* Each node shows table name

**Edges (dependencies):**
* **Arrows** show data flow
* Bronze → Silver → Gold
* Automatic dependency detection

**Node colors/status:**
* 🟢 **Green** - Completed successfully
* 🔵 **Blue** - Currently running
* 🟡 **Yellow** - Queued/waiting
* 🔴 **Red** - Failed
* ⚪ **Gray** - Not started

**Interactive graph:**
* Click on a node to see details
* Zoom in/out
* Pan around
* Auto-layout

**Try this:**
1. Watch nodes change color as pipeline runs
2. Click on a node to see:
   * Table schema
   * Row count
   * Execution time
   * Data quality metrics

✅ **Checkpoint:** Can view and understand pipeline graph

---

**💡 Tip:** The graph is your pipeline documentation - shows data lineage visually

### 📊 Step 6: Monitor Pipeline Execution

**Follow these steps:**

While the pipeline runs:

1. **Watch the graph:**
   * Nodes turn blue (running) then green (completed)
   * Progress flows from bronze → silver → gold

2. **Check execution details:**
   * Click on a running/completed node
   * Side panel shows:
     - **Status** - Running, completed, failed
     - **Duration** - How long it took
     - **Rows processed** - Number of records
     - **Data quality** - Pass/fail metrics (if expectations exist)

3. **View event log:**
   * Click **"Event log"** tab
   * Shows detailed execution logs
   * Timestamps for each step
   * Error messages (if any)

**Pipeline execution order:**
```
1. Bronze tables (parallel)
   ↓
2. Silver tables (after bronze completes)
   ↓
3. Silver enriched (after silver completes)
   ↓
4. Gold views (after silver completes)
```

**Execution states:**
* 🟡 **Queued** - Waiting for dependencies
* 🔵 **Running** - Currently processing
* 🟢 **Completed** - Successfully finished
* 🔴 **Failed** - Error occurred

✅ **Checkpoint:** Pipeline execution monitored

---

**💡 Tip:** Event log is crucial for debugging - shows detailed execution info

### 📊 Step 7: View Pipeline Results

**Follow these steps:**

After pipeline completes:

1. **Check final status:**
   * All nodes should be green
   * Status shows "Succeeded"
   * Total execution time displayed

2. **View table details:**
   * Click on any node in the graph
   * Side panel shows:
     - **Row count** - Number of records
     - **Schema** - Column names and types
     - **Sample data** - Preview of data
     - **Lineage** - Upstream and downstream tables

3. **Query the tables:**
   * Tables are created in your target schema
   * Open a SQL query or notebook
   * Query the tables:
   ```sql
   -- Bronze
   SELECT * FROM main.default.bronze_bookings LIMIT 10;
   
   -- Silver
   SELECT * FROM main.default.silver_bookings LIMIT 10;
   
   -- Gold
   SELECT * FROM main.default.gold_daily_bookings;
   ```

4. **Verify data flow:**
   * Bronze has raw data
   * Silver has cleaned data (fewer rows)
   * Gold has aggregated data (much fewer rows)

✅ **Checkpoint:** Can query pipeline output tables

---

**💡 Tip:** DLT tables are regular Delta tables - query them like any other table

## 🎯 Practice Challenges

Now it's your turn! Complete these challenges to reinforce your learning.

---

### 🎯 Challenge 1: Add a New Bronze Table

**Your task:**

Add a bronze table for the users data.

**Requirements:**
1. Create a streaming table named `bronze_users`
2. Ingest from `samples.wanderbricks.users`
3. Add appropriate comment
4. Use `@dlt.table` decorator
5. Use `spark.readStream.table()`

**Expected code structure:**
```python
@dlt.table(
    name="bronze_users",
    comment="Your comment here"
)
def bronze_users():
    return spark.readStream.table("samples.wanderbricks.users")
```

**After adding:**
1. Save the notebook
2. Go to pipeline UI
3. Click "Start" to run
4. Verify new table appears in graph

✅ **Success criteria:** bronze_users table created and visible in pipeline graph

---

### 🎯 Challenge 2: Create Silver Users Table

**Your task:**

Create a cleaned silver table for users.

**Requirements:**
1. Create streaming table named `silver_users`
2. Read from `bronze_users` using `dlt.read_stream()`
3. Apply data quality filters:
   * Filter out null emails
   * Filter out null names
4. Add calculated column:
   * `account_age_days` = DATEDIFF(CURRENT_DATE(), created_at)
5. Select relevant columns

**Hint:**
```python
@dlt.table(
    name="silver_users",
    comment="Cleaned user data - Silver layer"
)
def silver_users():
    return (
        dlt.read_stream("bronze_users")
        .filter(col("email").isNotNull())
        .filter(col("name").isNotNull())
        .withColumn("account_age_days", datediff(current_date(), col("created_at")))
        .select("user_id", "email", "name", "country", "user_type", 
                "created_at", "account_age_days")
    )
```

**After adding:**
1. Save notebook
2. Run pipeline
3. Verify silver_users appears in graph
4. Check it connects to bronze_users

✅ **Success criteria:** silver_users table with data quality filters

---

### 🎯 Challenge 3: Create Gold Metrics View

**Your task:**

Create a gold materialized view with user metrics.

**Requirements:**
1. Create materialized view named `gold_user_metrics`
2. Use `@dlt.view` decorator (not @dlt.table)
3. Read from `silver_users` using `dlt.read()` (not read_stream)
4. Group by `country` and `user_type`
5. Calculate:
   * Total user count
   * Average account age
   * Business user count
6. Order by total users descending

**Hint:**
```python
@dlt.view(
    name="gold_user_metrics",
    comment="User metrics by country and type - Gold layer"
)
def gold_user_metrics():
    return (
        dlt.read("silver_users")
        .groupBy("country", "user_type")
        .agg(
            count("user_id").alias("total_users"),
            avg("account_age_days").alias("avg_account_age_days"),
            sum(when(col("is_business") == True, 1).otherwise(0)).alias("business_users")
        )
        .orderBy(desc("total_users"))
    )
```

**After adding:**
1. Save notebook
2. Run pipeline
3. Verify gold_user_metrics appears
4. Query the view to see results

✅ **Success criteria:** gold_user_metrics view with aggregated data

---

### 🎯 Challenge 4: Build Complete End-to-End Flow

**Your task:**

Create a complete bronze → silver → gold flow for a new entity.

**Requirements:**

Build a pipeline for **page views** data:

**Bronze:**
```python
@dlt.table(name="bronze_page_views", comment="Raw page view data")
def bronze_page_views():
    return spark.readStream.table("samples.wanderbricks.page_views")
```

**Silver:**
```python
@dlt.table(name="silver_page_views", comment="Cleaned page views")
def silver_page_views():
    return (
        dlt.read_stream("bronze_page_views")
        .filter(col("user_id").isNotNull())
        .filter(col("property_id").isNotNull())
        .withColumn("view_date", to_date(col("timestamp")))
        .withColumn("view_hour", hour(col("timestamp")))
        .select("view_id", "user_id", "property_id", "page_url", 
                "device_type", "referrer", "view_date", "view_hour", "timestamp")
    )
```

**Gold:**
```python
@dlt.view(name="gold_page_view_metrics", comment="Page view metrics")
def gold_page_view_metrics():
    return (
        dlt.read("silver_page_views")
        .groupBy("view_date", "device_type")
        .agg(
            count("view_id").alias("total_views"),
            countDistinct("user_id").alias("unique_users"),
            countDistinct("property_id").alias("properties_viewed")
        )
        .orderBy("view_date", "device_type")
    )
```

**After adding:**
1. Add all three tables to the notebook
2. Save notebook
3. Run pipeline
4. Verify all 3 tables appear in graph
5. Check dependencies are correct
6. Query gold table to see metrics

✅ **Success criteria:** Complete bronze → silver → gold flow for page views

---

## 📚 Lakeflow Best Practices

### **Table Type Selection**

**Use Streaming Tables when:**
* ✅ Bronze layer (always)
* ✅ Silver layer (usually)
* ✅ Append-only data
* ✅ Large data volumes
* ✅ Need low latency
* ✅ Incremental processing

**Use Materialized Views when:**
* ✅ Gold layer aggregations
* ✅ Complex joins
* ✅ Small result sets
* ✅ Full refresh is acceptable
* ✅ Aggregations with GROUP BY

---

### **Code Organization**

**Structure your notebook:**
```python
# 1. Imports at top
import dlt
from pyspark.sql.functions import *

# 2. Bronze layer
@dlt.table
def bronze_table1(): ...

@dlt.table
def bronze_table2(): ...

# 3. Silver layer
@dlt.table
def silver_table1(): ...

# 4. Gold layer
@dlt.view
def gold_view1(): ...
```

**Naming conventions:**
* ✅ Prefix with layer: `bronze_`, `silver_`, `gold_`
* ✅ Use descriptive names: `silver_bookings_enriched`
* ✅ Be consistent across pipeline

---

### **Data Quality**

**Bronze layer:**
* ✅ No filtering (keep all data)
* ✅ Add metadata columns if needed
* ✅ Preserve raw data

**Silver layer:**
* ✅ Filter invalid data
* ✅ Remove nulls where appropriate
* ✅ Validate business rules
* ✅ Add calculated columns
* ✅ Deduplicate if needed

**Gold layer:**
* ✅ Aggregate clean data
* ✅ Create business metrics
* ✅ Optimize for queries

---

### **Performance**

**Optimize streaming tables:**
* ✅ Use appropriate partition columns
* ✅ Limit data scanned with filters
* ✅ Use broadcast joins for small dimensions
* ✅ Enable Photon acceleration

**Optimize materialized views:**
* ✅ Aggregate early
* ✅ Use efficient joins
* ✅ Limit result set size
* ✅ Use appropriate data types

---

### **Pipeline Configuration**

**Development:**
* ✅ Use Triggered mode
* ✅ Smaller cluster (1-2 workers)
* ✅ Development channel

**Production:**
* ✅ Use Continuous mode (if real-time needed)
* ✅ Enhanced autoscaling
* ✅ Serverless (if available)
* ✅ Current channel
* ✅ Enable monitoring

---

## 💡 Key Concepts Recap

### **Declarative vs Imperative**

**Imperative (Traditional):**
```python
# You manage everything
df = spark.read.table("source")
df_cleaned = df.filter(...)
df_cleaned.write.mode("append").saveAsTable("target")
# Handle checkpoints, retries, dependencies manually
```

**Declarative (Lakeflow):**
```python
# You declare what you want
@dlt.table
def target():
    return dlt.read_stream("source").filter(...)
# Lakeflow handles everything else automatically
```

---

### **Automatic Features**

**Lakeflow automatically handles:**
* ✅ **Dependencies** - Runs tables in correct order
* ✅ **Incremental processing** - Only processes new data
* ✅ **Checkpoints** - Tracks progress
* ✅ **Retries** - Handles failures
* ✅ **Scaling** - Adjusts compute resources
* ✅ **Monitoring** - Tracks metrics
* ✅ **Lineage** - Shows data flow

---

### **Reading Data in DLT**

**From source tables:**
```python
# Streaming
spark.readStream.table("source")

# Batch
spark.read.table("source")
```

**From DLT tables:**
```python
# Streaming (for streaming tables)
dlt.read_stream("upstream_table")

# Batch (for materialized views)
dlt.read("upstream_table")
```

---

### **Medallion Architecture in DLT**

```
🥉 Bronze (Streaming Tables)
   - Raw data ingestion
   - spark.readStream.table()
   - No transformations
   - Append-only
   ↓
🥈 Silver (Streaming Tables)
   - Data cleaning
   - dlt.read_stream()
   - Filters and validations
   - Calculated columns
   ↓
🥇 Gold (Materialized Views)
   - Aggregations
   - dlt.read()
   - Business metrics
   - Dashboard-ready
```

---

## 📊 Complete Pipeline Overview

**Your pipeline now has:**

### **Bronze Layer (3 streaming tables)**
* `bronze_bookings` - Raw bookings
* `bronze_properties` - Raw properties
* `bronze_reviews` - Raw reviews

### **Silver Layer (4 streaming tables)**
* `silver_bookings` - Cleaned bookings
* `silver_properties` - Cleaned properties
* `silver_reviews` - Cleaned reviews
* `silver_bookings_enriched` - Bookings + property details

### **Gold Layer (3 materialized views)**
* `gold_daily_bookings` - Daily metrics by property type
* `gold_property_performance` - Property type performance
* `gold_review_summary` - Review ratings over time

**Plus challenges (if completed):**
* `bronze_users`, `silver_users`, `gold_user_metrics`
* `bronze_page_views`, `silver_page_views`, `gold_page_view_metrics`

---

### **Data Flow:**

```
samples.wanderbricks.bookings
    ↓ (readStream)
bronze_bookings
    ↓ (read_stream + filter)
silver_bookings
    ↓ (read_stream + join)
silver_bookings_enriched
    ↓ (read + groupBy)
gold_daily_bookings
```

---

### **Pipeline Characteristics:**

* ✅ **10+ tables/views** (base + challenges)
* ✅ **Automatic dependencies** - Lakeflow manages order
* ✅ **Incremental processing** - Streaming tables
* ✅ **Data quality** - Filters and validations
* ✅ **Business metrics** - Gold aggregations
* ✅ **Visual lineage** - Graph shows flow

---

## 🔍 Querying Your Pipeline Results

**After pipeline completes, query the tables:**

### **Bronze tables:**
```sql
-- Raw data (all records)
SELECT COUNT(*) FROM main.default.bronze_bookings;
SELECT * FROM main.default.bronze_bookings LIMIT 10;
```

### **Silver tables:**
```sql
-- Cleaned data (fewer records)
SELECT COUNT(*) FROM main.default.silver_bookings;
SELECT * FROM main.default.silver_bookings_enriched LIMIT 10;
```

### **Gold views:**
```sql
-- Aggregated metrics
SELECT * FROM main.default.gold_daily_bookings 
ORDER BY booking_date DESC 
LIMIT 30;

SELECT * FROM main.default.gold_property_performance;

SELECT * FROM main.default.gold_review_summary 
WHERE review_year = 2024;
```

---

### **Verify data quality:**
```sql
-- Compare row counts
SELECT 'Bronze' AS layer, COUNT(*) AS row_count 
FROM main.default.bronze_bookings
UNION ALL
SELECT 'Silver' AS layer, COUNT(*) AS row_count 
FROM main.default.silver_bookings
UNION ALL
SELECT 'Gold' AS layer, COUNT(*) AS row_count 
FROM main.default.gold_daily_bookings;
```

**Expected results:**
* Bronze: Most rows (all data)
* Silver: Fewer rows (filtered)
* Gold: Much fewer rows (aggregated)

---

**💡 Tip:** Use these tables in dashboards, queries, and downstream applications

## 🔄 Updating Your Pipeline

**How to make changes:**

### **Step 1: Edit the notebook**
1. Open this notebook
2. Add, modify, or remove `@dlt.table` or `@dlt.view` definitions
3. Save the notebook

### **Step 2: Run the pipeline**
1. Go to the pipeline UI
2. Click **"Start"**
3. Lakeflow detects changes automatically
4. Updates affected tables

**What Lakeflow does:**
* ✅ Detects new tables (adds them)
* ✅ Detects modified tables (updates them)
* ✅ Detects removed tables (keeps them, doesn't delete)
* ✅ Updates dependencies automatically
* ✅ Processes only affected tables

---

### **Common updates:**

**Add a new table:**
```python
@dlt.table
def new_table():
    return dlt.read_stream("source").filter(...)
```
* Save notebook
* Run pipeline
* New table appears in graph

**Modify transformation:**
```python
@dlt.table
def silver_bookings():
    return (
        dlt.read_stream("bronze_bookings")
        .filter(col("status") == "confirmed")
        .filter(col("total_amount") > 100)  # NEW FILTER
    )
```
* Save notebook
* Run pipeline
* Table is reprocessed with new logic

**Add calculated column:**
```python
.withColumn("new_column", expr("calculation"))
```
* Save notebook
* Run pipeline
* Schema updates automatically

---

**💡 Tip:** Lakeflow handles schema evolution automatically

## 🔧 Troubleshooting Common Issues

**Pipeline won't start:**
* ✅ Check notebook is saved
* ✅ Verify notebook path in pipeline settings
* ✅ Ensure target schema exists
* ✅ Check permissions on source tables
* ✅ Verify SQL warehouse or cluster is available

**Table shows as failed (red):**
* ✅ Click the node to see error message
* ✅ Check Event log for details
* ✅ Common issues:
   - Syntax errors in code
   - Invalid column references
   - Source table doesn't exist
   - Permission issues
* ✅ Fix code and rerun

**Dependencies not detected:**
* ✅ Use `dlt.read()` or `dlt.read_stream()` (not spark.read)
* ✅ Check table names match exactly
* ✅ Verify decorator syntax is correct

**Streaming table not processing incrementally:**
* ✅ Verify using `readStream` (not `read`)
* ✅ Check source supports streaming
* ✅ Ensure checkpoint location is accessible

**Materialized view taking too long:**
* ✅ Check if aggregating too much data
* ✅ Add filters to reduce data volume
* ✅ Consider using streaming table instead
* ✅ Optimize source queries

**Schema evolution issues:**
* ✅ DLT handles schema evolution automatically
* ✅ Check if column types are compatible
* ✅ Review schema in table details

---

**💡 Tip:** Event log is your best friend for debugging - shows detailed error messages

## 📊 Quick Reference Guide

### **Decorators**
```python
# Streaming table
@dlt.table(name="table_name", comment="Description")
def table_name():
    return spark.readStream.table("source")

# Materialized view
@dlt.view(name="view_name", comment="Description")
def view_name():
    return dlt.read("source").groupBy(...).agg(...)
```

---

### **Reading Data**
```python
# From source (streaming)
spark.readStream.table("catalog.schema.table")

# From source (batch)
spark.read.table("catalog.schema.table")

# From DLT table (streaming)
dlt.read_stream("dlt_table_name")

# From DLT table (batch)
dlt.read("dlt_table_name")
```

---

### **Common Patterns**
```python
# Bronze: Ingest raw
@dlt.table
def bronze_table():
    return spark.readStream.table("source")

# Silver: Clean and transform
@dlt.table
def silver_table():
    return (
        dlt.read_stream("bronze_table")
        .filter(col("column").isNotNull())
        .withColumn("new_col", expr("calculation"))
    )

# Gold: Aggregate
@dlt.view
def gold_metrics():
    return (
        dlt.read("silver_table")
        .groupBy("dimension")
        .agg(count("*").alias("total"))
    )
```

---

### **Pipeline UI**
* **Create:** Workflows → Delta Live Tables → Create Pipeline
* **Configure:** Set notebook path, target schema, compute
* **Run:** Click Start button
* **Monitor:** Graph tab shows visual lineage
* **Debug:** Event log shows execution details
* **Query:** Tables created in target schema

---

### **Layer Guidelines**

| Layer | Table Type | Read Method | Purpose |
|-------|-----------|-------------|----------|
| Bronze | Streaming | readStream | Raw ingestion |
| Silver | Streaming | read_stream | Cleaning |
| Gold | Materialized | read | Aggregations |

---

## 🎉 Congratulations!

You've completed the Lakeflow Pipeline Fundamentals demo!

### **What You Learned:**

✅ **Lakeflow concepts** - Declarative pipeline framework  
✅ **Streaming tables** - Incremental processing  
✅ **Materialized views** - Full refresh aggregations  
✅ **Bronze layer** - Raw data ingestion  
✅ **Silver layer** - Data cleaning and validation  
✅ **Gold layer** - Business metrics  
✅ **Pipeline creation** - UI configuration  
✅ **Pipeline execution** - Running and monitoring  
✅ **Data lineage** - Visual graph  
✅ **Best practices** - Production patterns  

---

### **Key Takeaways:**

1. **Declarative is simpler** - Define what, not how
2. **Streaming tables for incremental** - Bronze and silver
3. **Materialized views for aggregations** - Gold layer
4. **Dependencies are automatic** - No manual orchestration
5. **Graph shows lineage** - Visual data flow
6. **Incremental by default** - Efficient processing
7. **Production-ready** - Built-in reliability

---

### **Your Pipeline:**

```
📁 samples.wanderbricks (source)
         ↓
🥉 Bronze Layer (3 streaming tables)
   - bronze_bookings
   - bronze_properties
   - bronze_reviews
         ↓
🥈 Silver Layer (4 streaming tables)
   - silver_bookings
   - silver_properties
   - silver_reviews
   - silver_bookings_enriched
         ↓
🥇 Gold Layer (3 materialized views)
   - gold_daily_bookings
   - gold_property_performance
   - gold_review_summary
         ↓
    📊 Dashboards & Analytics
```

---

### **What We Didn't Cover (Future Demos):**

* ⚠️ Expectations (data quality checks)
* 📅 Job scheduling
* 🔔 Alerting and monitoring
* 🔄 Change Data Capture (CDC)
* 🛡️ Advanced error handling
* 📊 Performance tuning
* 🔗 External data sources

---

### **Next Steps:**

* Build pipelines for your own data
* Explore expectations for data quality
* Learn about CDC patterns
* Set up production pipelines
* Create dashboards from gold tables
* Schedule pipeline runs
* Monitor pipeline health

---

### **Resources:**

* [Delta Live Tables Documentation](https://docs.databricks.com/delta-live-tables/index.html)
* [DLT Python API Reference](https://docs.databricks.com/delta-live-tables/python-ref.html)
* [DLT Best Practices](https://docs.databricks.com/delta-live-tables/best-practices.html)
* [Streaming Tables vs Views](https://docs.databricks.com/delta-live-tables/tables.html)

---

**You're now ready to build production data pipelines with Lakeflow!** 🚀

*Happy pipeline building!*